# GAN IMPLEMENTATION ON MNIST DATASET

In [1]:
# import necessary packages
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pylab
import h5py

from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow import keras

import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import random
from numpy.random import randint

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

from tqdm import tqdm 
from IPython import display 

In [2]:
# define the standalone discriminator model
def define_discriminator(in_shape=(200,200,1)):
	# weight initialization
	init = 'uniform'
	# define model
	model = Sequential()
	# downsample to 100x100
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 50x50
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [3]:
def define_generator(latent_dim):
	# weight initialization
	init = 'uniform'
	# define model
	model = Sequential()
	# foundation for 50x50 image
	n_nodes = 256 * 50 * 50
	model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((50, 50, 256)))
	# upsample to 100x100
	model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 200x200
	model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# output 28x28x1
	model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
	return model

In [4]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [5]:
# default values of standardised images
preferred_width = 200
preferred_height = 200

In [6]:
# paths to take images from and to save to (after transformation to BnW)
pathSource = "C:\\Users\\DataSci\\Desktop\\DS_Testing\\warGAN\\igTest"
pathDest = "C:\\Users\\DataSci\\Desktop\\DS_Testing\\warGAN\\BnWImages"
# list all objects in pathSource
dir_list = os.listdir(pathSource)

# setting a param for proportion of data to keep for testing
testingSize = int(len(dir_list)/5)

In [7]:
# create list, to contain all standardised images; values [0:1], instead of [0:255]
images = []

# for image in pathSource, convert them to BnW, save that to pathDest, then add standardised version to images[]
for image in dir_list:
    img = Image.open(pathSource+"\\"+image).convert('L').resize((preferred_width, preferred_height), Image.ANTIALIAS)
    # img.show()
    imgArray = np.array(img) # int array of pixels
    imgArray = imgArray.astype(float)/255 # standardise values to be 0:255
    images.append(imgArray)
    img.save(pathDest+"\\BnW"+image, "JPEG")

# diagnostic checks of images[]
print(np.shape(images))
print(images[0])
print(np.shape(images[0]))


(10, 200, 200)
[[0.4627451  0.4627451  0.4627451  ... 0.36470588 0.36470588 0.36470588]
 [0.4627451  0.4627451  0.4627451  ... 0.36470588 0.36470588 0.36470588]
 [0.46666667 0.46666667 0.46666667 ... 0.36470588 0.36470588 0.35294118]
 ...
 [0.4745098  0.47058824 0.4627451  ... 0.37647059 0.37647059 0.37647059]
 [0.46666667 0.47058824 0.45882353 ... 0.38823529 0.38039216 0.38431373]
 [0.4627451  0.47058824 0.4627451  ... 0.38431373 0.38823529 0.39215686]]
(200, 200)


In [8]:
X_testing = np.array(images[0:testingSize])
X_training = np.array(images[testingSize:len(images)])

print(np.shape(X_testing))
print(X_testing[1]) # checking it's correctly put something into the testing variable
# what we want to pass to the generator is list (X_testing[index])
print(np.shape(X_training))

(2, 200, 200)
[[0.53333333 0.53333333 0.53333333 ... 0.39607843 0.4        0.39607843]
 [0.53333333 0.53333333 0.53333333 ... 0.39607843 0.4        0.39607843]
 [0.5372549  0.5372549  0.53333333 ... 0.4        0.40392157 0.39607843]
 ...
 [0.43137255 0.41176471 0.42352941 ... 0.36470588 0.36078431 0.35686275]
 [0.41176471 0.42745098 0.42745098 ... 0.35294118 0.35294118 0.34901961]
 [0.4        0.43137255 0.41568627 ... 0.34901961 0.34901961 0.34901961]]
(8, 200, 200)


In [9]:
"""
train_data = tf.data.Dataset.from_tensor_slices((X_training))
valid_data = tf.data.Dataset.from_tensor_slices((X_testing))
"""

'\ntrain_data = tf.data.Dataset.from_tensor_slices((X_training))\nvalid_data = tf.data.Dataset.from_tensor_slices((X_testing))\n'

In [10]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return X, y

In [11]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = np.random.uniform(low=0.0, high=1.0, size=(latent_dim, n_samples))
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [12]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = abs(generate_latent_points(latent_dim, n_samples))
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n_samples, 1))
	return X, y

In [13]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=2):
	# calculate the number of batches per epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the total iterations based on batch and epoch
	n_steps = bat_per_epo * n_epochs
	# calculate the number of samples in half a batch
	half_batch = int(n_batch / 2)
	# prepare lists for storing stats each iteration
	d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# get randomly selected 'real' samples
		X_real, y_real = generate_real_samples(dataset, half_batch)
		# update discriminator model weights
		d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
		# generate 'fake' examples
		X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator model weights
		d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
		# prepare points in latent space as input for the generator
		X_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		# summarize loss on this batch
		print('>%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %
			(i+1, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
		# record history
		d1_hist.append(d_loss1)
		d2_hist.append(d_loss2)
		g_hist.append(g_loss)
		a1_hist.append(d_acc1)
		a2_hist.append(d_acc2)
		"""
		# evaluate the model performance every 'epoch'
		if (i+1) % bat_per_epo == 0:
			summarize_performance(i, g_model, latent_dim)
		"""
	print("finished training")
	g_model.save('WARgenerator.h5') # this is the generator that can be called externally

In [14]:
"""
set up the data into something the GAN can use in training
"""

X_train=np.reshape(X_training,(X_training.shape[0], X_training.shape[1],X_training.shape[2],1))

In [15]:
"""
set up for training and training call
this can take some time depending on number of epochs
recommend running on a GPU
"""

# size of the latent space
latent_dim = 50
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = X_train
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

>1, d1=0.663, d2=0.680 g=0.761, a1=100, a2=100
>2, d1=0.219, d2=1.011 g=0.491, a1=100, a2=0
>3, d1=0.137, d2=1.767 g=0.334, a1=100, a2=0
>4, d1=0.160, d2=1.500 g=0.539, a1=100, a2=0
>5, d1=0.565, d2=0.962 g=0.859, a1=100, a2=0
>6, d1=0.811, d2=0.690 g=0.978, a1=0, a2=100
>7, d1=0.962, d2=0.762 g=0.780, a1=0, a2=0
>8, d1=0.946, d2=0.843 g=0.638, a1=0, a2=0
>9, d1=0.938, d2=0.888 g=0.561, a1=0, a2=0
>10, d1=0.600, d2=1.034 g=0.474, a1=100, a2=0
>11, d1=0.584, d2=1.300 g=0.427, a1=100, a2=0
>12, d1=0.486, d2=1.311 g=0.414, a1=100, a2=0
>13, d1=0.535, d2=1.227 g=0.453, a1=100, a2=0
>14, d1=0.464, d2=1.191 g=0.438, a1=100, a2=0
>15, d1=0.489, d2=1.191 g=0.530, a1=100, a2=0
>16, d1=0.612, d2=0.999 g=0.636, a1=100, a2=0
>17, d1=0.614, d2=0.991 g=0.587, a1=100, a2=0
>18, d1=0.688, d2=0.961 g=0.578, a1=100, a2=0
>19, d1=0.588, d2=0.979 g=0.565, a1=100, a2=0
>20, d1=0.627, d2=0.958 g=0.597, a1=100, a2=0
>21, d1=0.487, d2=0.974 g=0.588, a1=100, a2=0
>22, d1=0.655, d2=0.974 g=0.646, a1=100, a2=0
>

In [16]:
"""
getting the model to draw something,
this can be done seperately, if put in another file
"""

model = load_model('WARgenerator.h5', compile = False)
latent_points = generate_latent_points(latent_dim, 1)
# generate images
predictionInstance = model.predict(latent_points) # has quite the think about it
print("done thinking")

done thinking


In [17]:
"""
getting the prediction into something useful
"""

xout = predictionInstance*255 # convert floats from ranges 0:1 to 0:255, so they can be blackness values in an image
# print(xout) # this is diagnostic
# np.shape(xout) # this is diagnostic

flatXOut = xout.flatten()  
# print(len(flatXOut)) # should be 40,000 # this is diagnostic
arr_2d = np.reshape(flatXOut, (200, 200))

array_int = abs(np.array(arr_2d, dtype='int')) # convert float array to ints
# print(np.shape(array_int)) # this is diagnostic
# array_int # this is diagnostic

In [18]:
# draw the image of the output array!!!!
img = Image.fromarray(array_int)
img.show()

In [19]:
print("Format: {0}\nSize: {1}\nMode: {2}".format(img.format, 
    img.size, img.mode)) 

Format: None
Size: (200, 200)
Mode: I


In [22]:
imgRGB = img.convert(mode="RGB")
imgRGB.save("output\\test.jpg")